# Run various tests on models

In [278]:
from game.game import Game
from game.data.generator import create_numbers
import torch
import os

In [508]:
# loads checkpoint and does one epoch, shows acc on validation set
def load(last_save, model_size, model_number, run_number, vocab, sender, receiver, embedding, length=1):
    epochs = last_save + 1
    save_name = "game/models/game{}_{}_{}_{}_{}_{}_{}".format(model_size, model_number, run_number, vocab, sender, receiver, embedding)
    if(length > 1):
        save_name = "{}_{}".format(save_name, length)
    

    params = [
        '--batch_size=1',
        '--n_epochs={}'.format(epochs),
        '--vocab_size={}'.format(vocab),
        '--sender_hidden={}'.format(sender),
        '--receiver_hidden={}'.format(receiver),
        '--lr=0.001',
        '--max_len={}'.format(length),
        '--sender_entropy=0.5',
        '--sender_embedding={}'.format(embedding),
        '--receiver_embedding={}'.format(embedding),
        '--validation_freq=10',
        '--train_data=game/data/train10.txt',
        '--validation_data=game/data/validation10.txt',
        '--load_from_checkpoint={}_{}.tar'.format(save_name, last_save)
    ]
    game = Game(params)
    game.play()
    return game

In [536]:
def predict_and_print(model, inputs, labels):
    inputs_tensor = torch.FloatTensor(inputs)
    labels_tensor = torch.FloatTensor(labels)
    prediction = model.game.forward(inputs_tensor,labels_tensor)
    messages = prediction[1].message.tolist()
    outputs = prediction[1].receiver_output.tolist()
    for q in range(len(outputs)):
        print("{} {} {} {}".format(inputs[q], messages[q], outputs[q], labels[q]))
    unique_outputs = set([item[0] for item in outputs])
    number_unique = len(unique_outputs)
    acc = prediction[1].aux['acc'].mean().item()
    print("Unique outputs: {}: {}".format(number_unique, unique_outputs))
    print("Accuracy: {}".format(acc))
    return acc, number_unique, outputs


In [292]:
extra_tests = {}
def test_above_limit(model, start, end, number_of_samples):
    # creates more samples for model to predict
    inputs, labels = extra_tests.get((start, end, number_of_samples), (None, None))
    if(inputs is None or labels is None):
        inputs, labels = create_numbers(end, number_of_samples, False, False, start)
        extra_tests[(start, end, number_of_samples)] = (inputs, labels)
        print("Inputs: {}".format(inputs))
        print("Labels: {}".format(labels))
    print("Testing {} samples of numbers that sum up to [{} - {}]".format(number_of_samples, start, end))
    return predict_and_print(game, inputs, labels)

In [293]:
def test_all_set(game, n):
    inputs = []
    labels = []
    for i in range(n+1):
        for j in range(n+1):
            if i+j <= n:
                inputs.append([i, j])
                labels.append([i+j]) 
    print("Testing {} samples of numbers that sum up to {}".format(len(inputs), n))
    return predict_and_print(game, inputs, labels)

In [341]:
def test_validation_only(game):
    frame = game.trainer.validation_data.dataset.frame
    inputs = [f[0].tolist() for f in frame]
    labels = [f[1].tolist() for f in frame]
    return predict_and_print(game, inputs, labels)

In [294]:
def test_some_other_numbers(game):
    inputs = [[9.0, 0.2], [0., 0.3],[6.0, 4.0],[6.0, 4.0],[9.0, 1.0],[9.0, 1.0],[9.0, 1.0],[1.0, 1.0],[1.0, 1.0],[1.0, 1.0], [0.0, 1.0], [0.0, 0.0], [-1.0, 4.0], [5.0, -2.0], [-1.0, 0.0], [-2.0, -2.0], [-9.0, -1.0], [8.0, -3.0], [-10.0, -10.0]]
    labels = [[9.2],[.3],[10.],[10.],[10.],[10.],[10.],[2.0],[2.0],[2.0],[1.0],[0.0],[3.0],[3.0], [-1.0], [-4.0], [-10.0], [5.0], [-20.0]]
    return predict_and_print(game, inputs, labels)

In [562]:
def test_synonymy(game, n, repeat):
    limit = n+1
    accs_input = {}
    messages_input = {}
    accs_label = {}
    messages_label = {}
    for i in range(limit):
        for j in range(limit):
            if(i+j <= n):
                inputs = []
                labels = []
                for p in range(repeat):
                    inputs.append([i, j])
                    labels.append([i+j]) 
                
                a, m, o = predict_and_print(game, inputs, labels)
                
                input = (inputs[0][0],inputs[0][1])
                label = labels[0][0]
                accs_input[input] = a
                messages_input[input] = m
                
                l = messages_label.get(label, [])
                l.append(o)
                messages_label[label] = l
                
                l = accs_label.get(label, [])
                l.append(a)
                accs_label[label] = l
                
    for i in range(limit):
        outputs = [output[0] for outputs in messages_label[i] for output in outputs]
        messages_label[i] = len(set(outputs))
        accs_label[i] = sum(accs_label[i])/len(accs_label[i])
        
    print("Unique messages per input: {}".format(messages_input))
    print("Accuracies per input: {}".format(accs_input))
    return accs_label, messages_label

In [362]:
def test_various_ranges(game, start_numbers, end_numbers, n_samples):
    accs = []
    ns_symbols = []
    for i in range(len(start_numbers)):  
        acc, n_symbols = test_above_limit(game, start_numbers[i], end_numbers[i], n_samples[i])
        accs.append(acc)
        ns_symbols.append(n_symbols)
    print("Accuracies: {}".format(accs))
    print("Number of messages: {}".format(ns_symbols))
    return accs, ns_symbols

In [394]:
def run_test(games, test):
    accs = []
    ns_symbols= []
    
    for i in range(len(games)):
        print("\n--------")
        print("Running test for game {}".format(i))
        print(games[i])
        print(games[i].game)
        acc, n_symbols = test(games[i])
        accs.append(acc)
        ns_symbols.append(n_symbols)
        
    print("\n--------")
    print("All games:")
    for i in range(len(accs)):
        print("Game {},  Number of messages: {}, Accuracy: {}".format(i, ns_symbols[i], accs[i]))

In [509]:
games10 = [
    load(1500, 10, 14, 1, 32, 64, 32, 7), # game10_14_1_32_64_32_7_1500.tar
    load(1500, 10, 14, 1, 64, 128, 64, 8), # game10_14_1_64_128_64_8_1500.tar
    load(1500, 10, 14, 1, 32, 32, 32, 7), #game10_14_1_32_32_32_7_1500.tar
    load(1500, 10, 14, 1, 64, 64, 64, 7), #game10_14_1_64_64_64_7_1500.tar
    load(1500, 10, 15, 1, 64, 64, 64, 7, 2), #game10_15_1_64_64_64_7_2_1500.tar
    load(6000, 10, 15, 2, 16, 16, 16, 5, 2)
]

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_14_1_32_64_32_7_1500.tar
# loading trainer state from game/models/game10_14_1_32_64_32_7_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=64, bias=True)
    )
    (hidden_to_output): Linear(in_features=64, out_features=32, bias=True)
    (embedding): Embedding(32, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 64)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=32, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 32, batch_first=True)
      (embedding): Embedding(32, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 8744

----------
Start of training... 2021-11-11 14:16:31.732372
{"loss": -0.211

# Test whole number range

In [383]:
run_test(games, lambda x: test_all_set(x, 10))


--------
Running test for game 0
Testing 66 samples of numbers that sum up to 10
[0, 0] [21, 0] 1.1616871356964111 [0]
[0, 1] [21, 0] 1.1616871356964111 [1]
[0, 2] [21, 0] 1.1616871356964111 [2]
[0, 3] [30, 0] 3.0647199153900146 [3]
[0, 4] [20, 0] 3.999807357788086 [4]
[0, 5] [9, 0] 5.029147624969482 [5]
[0, 6] [8, 0] 5.956601142883301 [6]
[0, 7] [12, 0] 6.933943271636963 [7]
[0, 8] [16, 0] 8.187851905822754 [8]
[0, 9] [17, 0] 9.367873191833496 [9]
[0, 10] [13, 0] 10.039083480834961 [10]
[1, 0] [21, 0] 1.1616871356964111 [1]
[1, 1] [21, 0] 1.1616871356964111 [2]
[1, 2] [30, 0] 3.0647199153900146 [3]
[1, 3] [20, 0] 3.999807357788086 [4]
[1, 4] [9, 0] 5.029147624969482 [5]
[1, 5] [8, 0] 5.956601142883301 [6]
[1, 6] [12, 0] 6.933943271636963 [7]
[1, 7] [16, 0] 8.187851905822754 [8]
[1, 8] [13, 0] 10.039083480834961 [9]
[1, 9] [13, 0] 10.039083480834961 [10]
[2, 0] [21, 0] 1.1616871356964111 [2]
[2, 1] [30, 0] 3.0647199153900146 [3]
[2, 2] [20, 0] 3.999807357788086 [4]
[2, 3] [9, 0] 5.029

# Test validation only

In [387]:
run_test(games, lambda x: test_validation_only(x))


--------
Running test for game 0
[5.0, 3.0] [16, 0] 8.187851905822754 [8.0]
[5.0, 2.0] [24, 0] 7.073504447937012 [7.0]
[0.0, 10.0] [13, 0] 10.039083480834961 [10.0]
[7.0, 0.0] [24, 0] 7.073504447937012 [7.0]
[5.0, 0.0] [9, 0] 5.029147624969482 [5.0]
[4.0, 0.0] [20, 0] 3.999807357788086 [4.0]
[0.0, 5.0] [9, 0] 5.029147624969482 [5.0]
[7.0, 3.0] [13, 0] 10.039083480834961 [10.0]
[1.0, 2.0] [30, 0] 3.0647199153900146 [3.0]
[1.0, 8.0] [13, 0] 10.039083480834961 [9.0]
[2.0, 8.0] [13, 0] 10.039083480834961 [10.0]
[2.0, 0.0] [21, 0] 1.1616871356964111 [2.0]
[4.0, 3.0] [24, 0] 7.073504447937012 [7.0]
[5.0, 1.0] [8, 0] 5.956601142883301 [6.0]
Unique outputs: 8: {1.1616871356964111, 3.999807357788086, 3.0647199153900146, 5.029147624969482, 5.956601142883301, 7.073504447937012, 8.187851905822754, 10.039083480834961}
Accuracy: 0.8571428656578064

--------
Running test for game 1
[5.0, 3.0] [38, 0] 8.05928897857666 [8.0]
[5.0, 2.0] [3, 0] 7.160861492156982 [7.0]
[0.0, 10.0] [25, 0] 9.8539876937866

# Test above number range

In [389]:
start_numbers = [11, 13, 50, 101]
end_numbers = [12, 20, 100, 100000]
n_samples = [10, 100, 100, 1000]
run_test(games, lambda x: test_various_ranges(x, start_numbers, end_numbers, n_samples))


--------
Running test for game 0
Testing 10 samples of numbers that sum up to [11 - 12]
[5, 7] [1, 1, 0] 10.000516891479492 [12]
[6, 5] [1, 1, 0] 10.000516891479492 [11]
[0, 11] [1, 1, 0] 10.000516891479492 [11]
[9, 2] [1, 1, 0] 10.000516891479492 [11]
[6, 6] [1, 1, 0] 10.000516891479492 [12]
[2, 9] [1, 1, 0] 10.000516891479492 [11]
[12, 0] [1, 1, 0] 10.000516891479492 [12]
[9, 3] [1, 1, 0] 10.000516891479492 [12]
[8, 4] [1, 1, 0] 10.000516891479492 [12]
[5, 6] [1, 1, 0] 10.000516891479492 [11]
Unique outputs: 1: {10.000516891479492}
Accuracy: 0.0
Testing 100 samples of numbers that sum up to [13 - 20]
[8, 8] [1, 1, 0] 10.000516891479492 [16]
[1, 19] [1, 1, 0] 10.000516891479492 [20]
[12, 5] [1, 1, 0] 10.000516891479492 [17]
[4, 16] [1, 1, 0] 10.000516891479492 [20]
[14, 4] [1, 1, 0] 10.000516891479492 [18]
[15, 4] [1, 1, 0] 10.000516891479492 [19]
[8, 7] [1, 1, 0] 10.000516891479492 [15]
[11, 6] [1, 1, 0] 10.000516891479492 [17]
[11, 5] [1, 1, 0] 10.000516891479492 [16]
[11, 3] [1, 1

[59774, 3230] [1, 1, 0] 10.000516891479492 [63004]
[45558, 3122] [1, 1, 0] 10.000516891479492 [48680]
[10892, 6001] [1, 1, 0] 10.000516891479492 [16893]
[16464, 63164] [1, 1, 0] 10.000516891479492 [79628]
[62073, 4612] [1, 1, 0] 10.000516891479492 [66685]
[7578, 282] [1, 1, 0] 10.000516891479492 [7860]
[17028, 20958] [1, 1, 0] 10.000516891479492 [37986]
[10978, 30970] [1, 1, 0] 10.000516891479492 [41948]
[343, 3116] [1, 1, 0] 10.000516891479492 [3459]
[21463, 15475] [1, 1, 0] 10.000516891479492 [36938]
[6442, 2600] [1, 1, 0] 10.000516891479492 [9042]
[6662, 3153] [1, 1, 0] 10.000516891479492 [9815]
[52732, 23190] [1, 1, 0] 10.000516891479492 [75922]
[269, 1760] [1, 1, 0] 10.000516891479492 [2029]
[23895, 22366] [1, 1, 0] 10.000516891479492 [46261]
[45287, 28817] [1, 1, 0] 10.000516891479492 [74104]
[32288, 37281] [1, 1, 0] 10.000516891479492 [69569]
[12675, 48487] [1, 1, 0] 10.000516891479492 [61162]
[10870, 14207] [1, 1, 0] 10.000516891479492 [25077]
[7845, 347] [1, 1, 0] 10.000516891

[69850, 24060] [1, 1, 0] 10.000516891479492 [93910]
[10027, 3509] [1, 1, 0] 10.000516891479492 [13536]
[28399, 37432] [1, 1, 0] 10.000516891479492 [65831]
[31949, 50469] [1, 1, 0] 10.000516891479492 [82418]
[77493, 2809] [1, 1, 0] 10.000516891479492 [80302]
[2085, 6614] [1, 1, 0] 10.000516891479492 [8699]
[32478, 13120] [1, 1, 0] 10.000516891479492 [45598]
[183, 3115] [1, 1, 0] 10.000516891479492 [3298]
[9331, 49692] [1, 1, 0] 10.000516891479492 [59023]
[66251, 33328] [1, 1, 0] 10.000516891479492 [99579]
[13673, 3966] [1, 1, 0] 10.000516891479492 [17639]
[61958, 1554] [1, 1, 0] 10.000516891479492 [63512]
[73416, 26017] [1, 1, 0] 10.000516891479492 [99433]
[67178, 9958] [1, 1, 0] 10.000516891479492 [77136]
[18386, 2875] [1, 1, 0] 10.000516891479492 [21261]
[472, 17828] [1, 1, 0] 10.000516891479492 [18300]
[6247, 70818] [1, 1, 0] 10.000516891479492 [77065]
[22967, 3118] [1, 1, 0] 10.000516891479492 [26085]
[7855, 14726] [1, 1, 0] 10.000516891479492 [22581]
[2902, 30555] [1, 1, 0] 10.0005

[31902, 49370] [1, 1, 0] 10.000516891479492 [81272]
[4569, 364] [1, 1, 0] 10.000516891479492 [4933]
[2982, 23429] [1, 1, 0] 10.000516891479492 [26411]
[46469, 24775] [1, 1, 0] 10.000516891479492 [71244]
[56957, 28565] [1, 1, 0] 10.000516891479492 [85522]
[7515, 17542] [1, 1, 0] 10.000516891479492 [25057]
[15339, 5350] [1, 1, 0] 10.000516891479492 [20689]
[38989, 47552] [1, 1, 0] 10.000516891479492 [86541]
[23128, 47874] [1, 1, 0] 10.000516891479492 [71002]
[23043, 62549] [1, 1, 0] 10.000516891479492 [85592]
[5763, 11263] [1, 1, 0] 10.000516891479492 [17026]
[4895, 3505] [1, 1, 0] 10.000516891479492 [8400]
[27081, 10513] [1, 1, 0] 10.000516891479492 [37594]
[10988, 6706] [1, 1, 0] 10.000516891479492 [17694]
[89723, 6702] [1, 1, 0] 10.000516891479492 [96425]
[59, 46155] [1, 1, 0] 10.000516891479492 [46214]
[18203, 41795] [1, 1, 0] 10.000516891479492 [59998]
[17408, 23635] [1, 1, 0] 10.000516891479492 [41043]
[38876, 32820] [1, 1, 0] 10.000516891479492 [71696]
[27602, 65635] [1, 1, 0] 10.

[7536, 5797] [1, 1, 0] 10.000516891479492 [13333]
[34662, 5020] [1, 1, 0] 10.000516891479492 [39682]
[31072, 974] [1, 1, 0] 10.000516891479492 [32046]
[11244, 33712] [1, 1, 0] 10.000516891479492 [44956]
[8440, 4261] [1, 1, 0] 10.000516891479492 [12701]
[34716, 50038] [1, 1, 0] 10.000516891479492 [84754]
[21035, 72398] [1, 1, 0] 10.000516891479492 [93433]
[2933, 5874] [1, 1, 0] 10.000516891479492 [8807]
[56193, 15191] [1, 1, 0] 10.000516891479492 [71384]
[57498, 30616] [1, 1, 0] 10.000516891479492 [88114]
[16378, 2661] [1, 1, 0] 10.000516891479492 [19039]
[11525, 41822] [1, 1, 0] 10.000516891479492 [53347]
[14001, 79031] [1, 1, 0] 10.000516891479492 [93032]
[42263, 51812] [1, 1, 0] 10.000516891479492 [94075]
[21955, 10982] [1, 1, 0] 10.000516891479492 [32937]
[193, 115] [1, 1, 0] 10.000516891479492 [308]
[6507, 27973] [1, 1, 0] 10.000516891479492 [34480]
[3876, 6734] [1, 1, 0] 10.000516891479492 [10610]
[20169, 32738] [1, 1, 0] 10.000516891479492 [52907]
[10346, 35946] [1, 1, 0] 10.0005

# Test various numbers

In [390]:
run_test(games, lambda x: test_some_other_numbers(x))


--------
Running test for game 0
[9.0, 0.2] [13, 0] 10.039083480834961 [9.2]
[0.0, 0.3] [21, 0] 1.1616871356964111 [0.3]
[6.0, 4.0] [13, 0] 10.039083480834961 [10.0]
[6.0, 4.0] [13, 0] 10.039083480834961 [10.0]
[9.0, 1.0] [13, 0] 10.039083480834961 [10.0]
[9.0, 1.0] [13, 0] 10.039083480834961 [10.0]
[9.0, 1.0] [13, 0] 10.039083480834961 [10.0]
[1.0, 1.0] [21, 0] 1.1616871356964111 [2.0]
[1.0, 1.0] [21, 0] 1.1616871356964111 [2.0]
[1.0, 1.0] [21, 0] 1.1616871356964111 [2.0]
[0.0, 1.0] [21, 0] 1.1616871356964111 [1.0]
[0.0, 0.0] [21, 0] 1.1616871356964111 [0.0]
[-1.0, 4.0] [30, 0] 3.0647199153900146 [3.0]
[5.0, -2.0] [30, 0] 3.0647199153900146 [3.0]
[-1.0, 0.0] [21, 0] 1.1616871356964111 [-1.0]
[-2.0, -2.0] [21, 0] 1.1616871356964111 [-4.0]
[-9.0, -1.0] [21, 0] 1.1616871356964111 [-10.0]
[8.0, -3.0] [9, 0] 5.029147624969482 [5.0]
[-10.0, -10.0] [21, 0] 1.1616871356964111 [-20.0]
Unique outputs: 4: {1.1616871356964111, 10.039083480834961, 3.0647199153900146, 5.029147624969482}
Accuracy: 

# Test synonymy

In [563]:
number_range = 10
repeat_times = 100
run_test(games10, lambda x: test_synonymy(x, 10, 100))


--------
Running test for game 0
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=64, bias=True)
    )
    (hidden_to_output): Linear(in_features=64, out_features=32, bias=True)
    (embedding): Embedding(32, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 64)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=32, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 32, batch_first=True)
      (embedding): Embedding(32, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1.1861628293991089] [0]
[0, 0] [21, 0] [1

[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [8, 0] [5.9541802406311035] [5]
[1, 4] [8, 0] [5.9541802406311035] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [31, 0] [4.044685363769531] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.0004682540893555] [5]
[1, 4] [9, 0] [5.00046825

[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [8, 0] [5.9541802406311035] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [19, 0] [5.924770832061768] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.0004682540893555] [5]
[2, 3] [9, 0] [5.00046825

[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [23, 0] [7.969409942626953] [8]
[3, 5] [25, 0] [7.974661827087402] [8]
[3, 5] [2, 0] [9.279593467712402] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [25, 0] [7.974661827087402] [8]
[3, 5] [25, 0] [7.974661827087402] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [1, 0] [9.285616874694824] [8]
[3, 5] [11, 0] [7.973781585693359] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [14, 0] [8.025673866271973] [8]
[3, 5] [23, 0] [7.969409942626953] [8]
[3, 5] [22, 0] [8.015878677368164] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [23, 0] [7.969409942626953] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [25, 0] [7.974661827087402] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [14, 0] [8.025673866271973] [8]
[3, 5] [1, 0] [9.285616874694824] [8]
[3, 5] [16, 0] [8.072235107421875] [8]
[3, 5] [25, 0] [7.974661827087402] [8]
[3, 5] [25, 0] [7.974661827087402] [8]
[3, 5] [16, 0] [8.0722351074

[5, 2] [12, 0] [6.873895645141602] [7]
[5, 2] [27, 0] [6.982747554779053] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [12, 0] [6.873895645141602] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [6, 0] [6.97365665435791] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [6, 0] [6.97365665435791] [7]
[5, 2] [6, 0] [6.97365665435791] [7]
[5, 2] [0, 0] [7.02932071685791] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [12, 0] [6.873895645141602] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [0, 0] [7.02932071685791] [7]
[5, 2] [12, 0] [6.873895645141602] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [0, 0] [7.02932071685791] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [12, 0] [6.873895645141602] [7]
[5, 2] [0, 0] [7.02932071685791] [7]
[5, 2] [12, 0] [6.873895645141602] [7]
[5, 2] [24, 0] [7.014220237731934] [7]
[5, 2] [12, 0] [6.873895645141602] [7]


...

[2, 0] [7, 3, 0] [2.028193235397339] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [3, 12, 0] [2.0528416633605957] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [3, 11, 0] [2.0740489959716797] [2]
[2, 0] [3, 12, 0] [2.0528416633605957] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [3, 5, 0] [2.0603644847869873] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [3, 5, 0] [2.0603644847869873] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [7, 3, 0] [2.028193235397339] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [10, 3, 0] [2.022937297821045] [2]
[2, 0] [3, 2, 0] [1.720318078994751] [2]
[2, 0] [4, 3, 0] [2.015352725982666] [2]
[2, 0] [3, 9, 0] [2.052438974380493] [2]
[2, 0] [7, 3, 0] [2.028193235397339] [2]
[

[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [5, 3, 0] [5.007751941680908] [5]
[3, 2] [5, 3, 0] [5.007751941680908] [5]
[3, 2] [2, 4, 0] [4.9425811767578125] [5]
[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [12, 3, 0] [5.002150058746338] [5]
[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [9, 3, 0] [4.993997573852539] [5]
[3, 2] [12, 3, 0] [5.002150058746338] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [2, 0, 0] [4.857392311096191] [5]
[3, 2] [11, 3, 0] [4.995292663574219] [5]
[3, 2] [5, 3, 0] [5.007751941680908] [5]
[3, 2] [8, 3, 0] [5.022977828979492] [5]
[3, 2] [12, 3, 0] [5.002150058746338] [5]
[3, 2] [5, 3, 0] [5.007751941680908] [5]
[3, 2

[3, 7] [6, 14, 0] [9.927542686462402] [10]
[3, 7] [15, 1, 0] [9.955461502075195] [10]
[3, 7] [15, 1, 0] [9.955461502075195] [10]
[3, 7] [6, 6, 0] [9.946503639221191] [10]
[3, 7] [15, 1, 0] [9.955461502075195] [10]
[3, 7] [1, 13, 0] [9.998617172241211] [10]
[3, 7] [6, 13, 0] [9.944499015808105] [10]
[3, 7] [1, 13, 0] [9.998617172241211] [10]
[3, 7] [15, 1, 0] [9.955461502075195] [10]
[3, 7] [13, 1, 0] [9.953658103942871] [10]
[3, 7] [6, 14, 0] [9.927542686462402] [10]
[3, 7] [1, 6, 0] [10.009835243225098] [10]
[3, 7] [13, 1, 0] [9.953658103942871] [10]
[3, 7] [6, 14, 0] [9.927542686462402] [10]
[3, 7] [15, 1, 0] [9.955461502075195] [10]
[3, 7] [14, 15, 0] [9.962854385375977] [10]
[3, 7] [13, 1, 0] [9.953658103942871] [10]
[3, 7] [6, 6, 0] [9.946503639221191] [10]
[3, 7] [15, 1, 0] [9.955461502075195] [10]
[3, 7] [1, 1, 0] [10.014272689819336] [10]
[3, 7] [13, 1, 0] [9.953658103942871] [10]
[3, 7] [1, 5, 0] [9.976863861083984] [10]
[3, 7] [13, 1, 0] [9.953658103942871] [10]
[3, 7] [13, 1

[5, 5] [6, 15, 0] [9.973600387573242] [10]
[5, 5] [1, 15, 0] [9.996451377868652] [10]
[5, 5] [13, 1, 0] [9.953658103942871] [10]
[5, 5] [6, 13, 0] [9.944499015808105] [10]
[5, 5] [14, 14, 0] [9.908286094665527] [10]
[5, 5] [1, 14, 0] [10.009989738464355] [10]
[5, 5] [1, 13, 0] [9.998617172241211] [10]
[5, 5] [15, 1, 0] [9.955461502075195] [10]
[5, 5] [1, 15, 0] [9.996451377868652] [10]
[5, 5] [6, 14, 0] [9.927542686462402] [10]
[5, 5] [15, 1, 0] [9.955461502075195] [10]
[5, 5] [15, 1, 0] [9.955461502075195] [10]
[5, 5] [15, 1, 0] [9.955461502075195] [10]
[5, 5] [1, 12, 0] [9.969919204711914] [10]
[5, 5] [1, 13, 0] [9.998617172241211] [10]
[5, 5] [15, 1, 0] [9.955461502075195] [10]
[5, 5] [6, 13, 0] [9.944499015808105] [10]
[5, 5] [14, 1, 0] [9.96163272857666] [10]
[5, 5] [1, 5, 0] [9.976863861083984] [10]
[5, 5] [14, 6, 0] [9.931502342224121] [10]
[5, 5] [13, 1, 0] [9.953658103942871] [10]
[5, 5] [1, 11, 0] [9.979555130004883] [10]
[5, 5] [15, 1, 0] [9.955461502075195] [10]
[5, 5] [14,

[8, 1] [8, 1, 0] [8.990694999694824] [9]
[8, 1] [1, 4, 0] [8.96717643737793] [9]
[8, 1] [6, 12, 0] [8.968485832214355] [9]
[8, 1] [15, 13, 0] [9.002070426940918] [9]
[8, 1] [15, 14, 0] [8.944256782531738] [9]
[8, 1] [9, 1, 0] [8.97864055633545] [9]
[8, 1] [14, 5, 0] [8.97598648071289] [9]
[8, 1] [14, 12, 0] [8.961602210998535] [9]
[8, 1] [6, 15, 0] [9.973600387573242] [9]
[8, 1] [11, 1, 0] [8.981172561645508] [9]
[8, 1] [14, 5, 0] [8.97598648071289] [9]
[8, 1] [1, 7, 0] [8.961894035339355] [9]
[8, 1] [14, 8, 0] [8.9668550491333] [9]
[8, 1] [8, 1, 0] [8.990694999694824] [9]
[8, 1] [12, 1, 0] [8.982537269592285] [9]
[8, 1] [5, 1, 0] [8.983720779418945] [9]
[8, 1] [14, 5, 0] [8.97598648071289] [9]
[8, 1] [15, 6, 0] [8.959507942199707] [9]
[8, 1] [13, 13, 0] [8.93343734741211] [9]
[8, 1] [15, 6, 0] [8.959507942199707] [9]
[8, 1] [1, 7, 0] [8.961894035339355] [9]
[8, 1] [15, 6, 0] [8.959507942199707] [9]
[8, 1] [13, 15, 0] [8.948559761047363] [9]
[8, 1] [14, 5, 0] [8.97598648071289] [9]
[8,